Enter the timestack_name to be processed, this file must be in the same folder as this notebook. Choose the name of the model you want to use (tf_model_version should be Timestack_Shoreline_Xc_v0 for the model from the thesis) and associated parameters (for the thesis model use 64, 64, 10 and 0 for IMG_SIZE, IMG_SIZE_WID, exc_value, eve_value). Then run the cell (shift+enter). 

A GUI will open for setting the user defined parameters before automated tracing. By scrolling through the entire timestack, find the global maxima and global minima points. Also set the green slider to the height of 5 medium sized swash events from the top of the window. Once you've set the user defined parameters the best you can, simply close the window to start the automated swash tracing.

You will need to install tensorflow using 'pip install tensorflow' and cv2 using 'pip install opencv-python' using the anaconda prompt if they are not already installed.

In [ ]:
# this example timestack was created by Dr Hannah Power. The model has not been trained on data from this timestack.
timestack_name = "example_timestack.png"

# CNN model parameters
tf_model_version = "Timestack_Shoreline_Xc_v0"
IMG_SIZE = 64 # the size of image that the selected tensorflow model uses
IMG_SIZE_WID = 64 # sometimes this is different - if the image is taken as rectangular
exc_value = 10 # % excursion range increase or decrease
eve_value = 0 # % events height increase or decrease

# import libraries
import csv
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

# get the timestack image
timestack = cv2.imread(timestack_name)
height, width, _ = timestack.shape
padding = width

# initial slider values
min_uprush = 334
max_uprush = 828
five_peaks_height = 277
scroll_image = 0

"""%%%%%%%%%%%%%% GRAPHICAL USER INTERFACE %%%%%%%%%%%%%%"""

# setup GUI
title_window = "set the height of 5 peaks, the max runup point, the min rundown point"
timestack = cv2.imread(timestack_name)
timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

# show cursors
cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0], thickness=2)
cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
         thickness=2)
cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
         thickness=2)
timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
cv2.imshow(title_window, timestack_snapshot)

# setup sliders
def on_trackbar_min_uprush(val):
    global min_uprush

    min_uprush = val
    timestack = cv2.imread(timestack_name)
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show cursors
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0], thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

def on_trackbar_max_uprush(val):
    global max_uprush

    max_uprush = val
    timestack = cv2.imread(timestack_name)
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show cursors
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0],
             thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

def on_trackbar_five_peaks_height(val):
    global five_peaks_height

    five_peaks_height = val
    timestack = cv2.imread(timestack_name)
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show cursors
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0],
             thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

def on_trackbar_scroll_image(val):
    global scroll_image

    scroll_image = val
    timestack = cv2.imread(timestack_name)
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show cursors
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0],
             thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

# setup trackbar
trackbar_min_uprush = "Rd_min"
trackbar_max_uprush = "Ru_max"
trackbar_five_peaks_height = "L_5p"
trackbar_scroll_image = "Scroll"
cv2.createTrackbar(trackbar_min_uprush, title_window, min_uprush, padding, on_trackbar_min_uprush)
cv2.createTrackbar(trackbar_max_uprush, title_window, max_uprush, padding, on_trackbar_max_uprush)
cv2.createTrackbar(trackbar_five_peaks_height, title_window, five_peaks_height, padding, on_trackbar_five_peaks_height)
# only apply scroll trackbar if image doesn't fit on the screen
if height - padding > 0:
    cv2.createTrackbar(trackbar_scroll_image, title_window, scroll_image, height - padding, on_trackbar_scroll_image)
cv2.waitKey()

"""%%%%%%%%%%%%%% TIMESTACK PROCESSING %%%%%%%%%%%%%%"""

# setup timestack for processing
timestack = cv2.imread(timestack_name)
image_range = max_uprush - min_uprush
# add % to min and max uprush range if possible
if min_uprush - image_range*(exc_value/200) < 0:
    min_uprush = 0
else:
    min_uprush = int(min_uprush - image_range*(exc_value/200))
if max_uprush + image_range*(exc_value/200) > timestack.shape[1]:
    max_uprush = timestack.shape[1]
else:
    max_uprush = int(max_uprush + image_range*(exc_value/200))
timestack_focus = timestack[:, min_uprush:max_uprush]
timestack_annotated = timestack.copy()
height_of_two_peaks = int(five_peaks_height*(2/5)*(1 + eve_value/100))
padding = height_of_two_peaks
height, width, _ = timestack.shape
width = max_uprush - min_uprush
timestack_count = 0
tf_images = []
tf_model_xc = tf.keras.models.load_model(tf_model_version + ".model")
vertical_coordinates = []
horizontal_coordinates = []
tf_images_list = []
for i in range(0, int((height - padding)), 1):

    # get the snapshot of the image for the horizontal position i
    timestack_snapshot = timestack_focus[i:i + padding, :]

    # classify the timestack snapshot through the convolutional neural network for X_coordinate
    tf_images = cv2.resize(timestack_snapshot, (IMG_SIZE_WID, IMG_SIZE))
    tf_images_list.append(tf_images)

print("Processing timestack through CNN model")
# process all images through CNN model
tf_images_list = np.array(tf_images_list)
tf_images_list = tf.keras.utils.normalize(tf_images_list, axis=1)
tf_images_list = tf_images_list.reshape(-1, IMG_SIZE_WID, IMG_SIZE, 3)
predictions = tf_model_xc.predict([tf_images_list])

# save coordinates and annotate timestack
for i in range(0, int((height - padding)), 1):

    horizontal_coordinate = i + int(padding/2) # the row      
    vertical_coordinate = int((predictions[i][0])*width) + min_uprush # the column
    cv2.circle(timestack_annotated, (vertical_coordinate, horizontal_coordinate), radius=1, color=[255, 0, 0])
    vertical_coordinates.append(vertical_coordinate)
    horizontal_coordinates.append(horizontal_coordinate)
    timestack_count = timestack_count + 1

# create plot of swash front for a quick visual check
print("Red line is the swash front found by the CNN model")
plt.plot(vertical_coordinates, horizontal_coordinates, color = 'r')
plt.gca().invert_yaxis() # invert axis so it is like the timestack image coordinates
plt.xlabel('cross shore distance (pixels)')
plt.ylabel('time (pixels)')
plt.show()

# save the image
print("Saving image")
cv2.imwrite("annotated_runup_timestack_" + timestack_name[:-4] + "_" + tf_model_version + ".png", timestack_annotated)

# save the data
print("Saving data to csv file")

# create csv file
output_data_name = "runup_data_" + timestack_name[:-4] + "_" + tf_model_version + ".csv"
with open(output_data_name, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows([["timestack shoreline coordinates"]])

with open(output_data_name, 'a', newline='') as csvFile:
    writer = csv.writer(csvFile)
    time_row = list(["time (horizontal coordinate)"])
    time_row.extend(horizontal_coordinates)
    x_row = list(["x (vertical coordinate)"])
    x_row.extend(vertical_coordinates)
    writer.writerow(time_row)
    writer.writerow(x_row)

csvFile.close()

print("Done")